In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

In [2]:
import matplotlib
matplotlib.rcParams['figure.figsize'] = (10.0, 10.0)
matplotlib.rcParams['image.cmap'] = 'gray'

# Instructions

```

This was done on a Jupyter Notebook.

/////////////////////////////////////////////////////////////////////////////

INSTRUCTIONS

/////////////////////////////////////////////////////////////////////////////

** I've only tested this on a background image larger than the video image(in both width, and height)

- You can create use the scrollbar at the top to get the greenscreen color 

###OR 

you can shift + left mouse button on image to grab the pixel color

- Press 'Enter', also change the tolerance slider, 
the background won't change until a certain threshold is reached in the tolerance slider

```

In [3]:
path_vid = 'greenscreen-asteroid.mp4'
path_bg = 'free-hd-Ocean-Backgrounds.jpg'

# Global Setup
ini_window = 10 # setup the initial window size
k_timer = 25 # for waitKey(k_timer)
c_5x5 = cv2.getStructuringElement(cv2.MORPH_CROSS, (5,5)) # only need to make the kernel once
c_3x3 = cv2.getStructuringElement(cv2.MORPH_CROSS, (3,3)) # only need to make the kernel once

# Window Callback functions

In [4]:
# R trackbar event caller
def EventTrkChangeRed(*args):
  global r_val
  r_val = args[0]
  return

# G trackbar event caller
def EventTrkChangeGreen(*args):
  global g_val
  g_val = args[0]
  return

# B trackbar event caller
def EventTrkChangeBlue(*args):
  global b_val
  b_val = args[0]
  return

def EventTrkChangeTolerance(*args):
  global tol_val
  tol_val = args[0]
  return

def EventTrkChangeSoftness(*args):
  global soft_val
  soft_val = args[0]
  return

# Image Processing

In [10]:
'''
HSV Masking
'''
def UpdateImage(frame : np.array) -> np.array:

  global r_val, g_val, b_val, c_5x5, c_3x3, ini_window, bg
  
  px_bgr = np.zeros((1,1,3), np.uint8)
  px_bgr[0,0,0], px_bgr[0,0,1], px_bgr[0,0,2] = b_val, g_val, r_val
  px_hsv = cv2.cvtColor(px_bgr, cv2.COLOR_BGR2HSV)
  h_val, s_val, v_val = px_hsv[0,0,0], px_hsv[0,0,1], px_hsv[0,0,2]
  
# #   Debug
#   print(px_hsv)
#   print(h_val, s_val, v_val)
#   print('\n')
  
  frame_hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV).astype(np.uint8)
  row_frame, col_frame, dim = frame.shape
  row_bg, col_bg, _ = bg.shape
  
  lower_h, upper_h = max(0, h_val - ini_window - h_val*tol_val/200), min(180, h_val + ini_window + h_val*tol_val/200)
  lower_s, upper_s = max(0, s_val - ini_window - s_val*tol_val/200), min(255, s_val + ini_window + s_val*tol_val/200)
  lower_v, upper_v = max(0, v_val - ini_window - v_val*tol_val/200), min(255, v_val + ini_window + v_val*tol_val/200)
  
  mask_h = cv2.inRange(frame_hsv[:,:,0], lower_h, upper_h)
  mask_s = cv2.inRange(frame_hsv[:,:,1], lower_s, upper_s)
  mask_v = cv2.inRange(frame_hsv[:,:,2], lower_v, upper_v)
  mask = (mask_h & mask_s & mask_v) == 255

  min_row, min_col = min(row_bg, row_frame), min(col_bg, col_frame)
  cropped_bg = bg[:min_row, :min_col, :]
  cropped_mask = mask[:min_row, :min_col]
  frame[mask == True] = cropped_bg[mask]
#   frame[mask[:min_row, :min_col] == True] = bg[:min_row,:min_col, :][mask[:min_row, :min_col]]

  return frame

# Mouse Event

In [6]:
# on Shift + left mouse button
# get the color sample at x, y
# then change the HSV bar to equal the resulting sample color
def onMouseEvent(event, x, y, flags, params):
  global r_val, g_val, b_val, frame_is_none
  if event == cv2.EVENT_LBUTTONDOWN and flags == cv2.EVENT_FLAG_SHIFTKEY + 1:
    if frame is None:
      print('Null frame')
      return
#     print((y,x), frame.shape)
    px = frame[y:y+1,x:x+1]
#     print(px)
    r_val, g_val, b_val = px[0][0]
  return

# Features

+ 2 features required : color sampler, tolerance slider(take mean of RGB take the percentage difference as an ```inrange``` window)
+ 2 features optional : softness slider, color cast
+ softness slider implementation: Blur original image. perform distance calculation on binary (do once) -> binary threshold on distance image(perform every trackbar change)

### Code for video loop
```
frame_counter += 1
#If the last frame is reached, reset the capture and the frame_counter
if frame_counter == cap.get(cv2.cv.CV_CAP_PROP_FRAME_COUNT):
    frame_counter = 0 #Or whatever as long as it is the same as next line
    cap.set(cv2.cv.CV_CAP_PROP_POS_FRAMES, 0)
```

In [12]:
cap = cv2.VideoCapture(path_vid)

if(not cap.isOpened()):
  print('Something went wrong with the video')
else:
  # initial setup
  bg = cv2.imread(path_bg)
  k = 0 # reads keyboard inputs
  isConverting = False # True -> perform chroma keying, False -> don't perform chroma_keying
  frame_counter = 0
  cal_win = 'Calibration Window'
  r_txt, g_txt, b_txt = 'Red', 'Green', 'Blue'
  r_val, r_max = 0, 255
  g_val, g_max = 0, 255
  b_val, b_max = 0, 255
  tol_txt, tol_val, tol_max = 'Tolerance(%)', 0, 100
  soft_txt, soft_val, soft_max = 'Softness(%)', 0, 100
  
  while cap.isOpened() and k != 27:
    frame_counter += 1
    if k == ord('r'):
      frame_counter = 0
      cap.set(cv2.CAP_PROP_POS_FRAMES, 0)
    ret, frame = cap.read()
    cv2.namedWindow(cal_win, cv2.WINDOW_NORMAL)
    cv2.createTrackbar(r_txt, cal_win, r_val, r_max, EventTrkChangeRed)
    cv2.createTrackbar(g_txt, cal_win, g_val, g_max, EventTrkChangeGreen)
    cv2.createTrackbar(b_txt, cal_win, b_val, b_max, EventTrkChangeBlue)
    cv2.createTrackbar(tol_txt, cal_win, tol_val, tol_max, EventTrkChangeTolerance)
#     cv2.createTrackbar(soft_txt, cal_win, soft_val, soft_max, EventTrkChangeSoftness)
    cv2.setMouseCallback(cal_win, onMouseEvent)
    if k == 13: # enter key
      isConverting = ~isConverting
    if ret:
      new_frame = None
      if isConverting:
        k_timer = 5
        new_frame = UpdateImage(frame)
      else:
        k_timer = 25
        new_frame = frame
      cv2.imshow(cal_win, new_frame)
    k = cv2.waitKey(k_timer) & 0xff # read user keyboard input
cap.release()
cv2.destroyAllWindows()